# Agenda

1. Pivot tables
2. Stack, unstack, melt
3. Joins
4. `filter` on columns
5. Cleaning data

In [1]:
import numpy as np
import pandas as pd 
from pandas import Series, DataFrame

In [2]:
df = DataFrame(np.random.randint(0, 1000, [4,5]),
              index=list('abcd'),
              columns=list('vwxyz'))
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466


In [3]:
df.loc['e'] = [1,2,3,4,5]
df

,v,w,x,y,z
a,906,704,539,445,618
b,692,477,543,649,594
c,853,732,548,193,643
d,501,406,614,962,466
e,1,2,3,4,5


In [4]:
new_row = {'w':20, 'x':30, 'z':50}
df.loc['f'] = new_row

In [5]:
df

,v,w,x,y,z
a,906.0,704,539,445.0,618
b,692.0,477,543,649.0,594
c,853.0,732,548,193.0,643
d,501.0,406,614,962.0,466
e,1.0,2,3,4.0,5
f,NaN,20,30,NaN,50


In [6]:
df['w'] = [10, 20, 30, 40, 50, 60]
df

,v,w,x,y,z
a,906.0,10,539,445.0,618
b,692.0,20,543,649.0,594
c,853.0,30,548,193.0,643
d,501.0,40,614,962.0,466
e,1.0,50,3,4.0,5
f,NaN,60,30,NaN,50


In [7]:
df['u'] = {'a':100, 'c':300, 'e':500}

In [8]:
df

,v,w,x,y,z,u
a,906.0,10,539,445.0,618,100.0
b,692.0,20,543,649.0,594,NaN
c,853.0,30,548,193.0,643,300.0
d,501.0,40,614,962.0,466,NaN
e,1.0,50,3,4.0,5,500.0
f,NaN,60,30,NaN,50,NaN


# Grouping

If we have a data frame with:
- One categorical column
- One numeric column

We can use `groupby` to calculate an aggregation method once per category, for all numeric rows matching.

In [9]:
df = pd.read_csv('taxi.csv')
df.groupby('passenger_count')['total_amount'].mean()

passenger_count
0    25.570000
1    17.368569
2    18.406306
3    17.994704
4    18.881648
5    17.211269
6    17.401355
Name: total_amount, dtype: float64

In [10]:
# we can do a 2D grouping

df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

# Pivot table

A pivot table is a 2-dimensional grouping!  It's similar to our multi-index, but uses a table

- We need one categorical column -- this will be the index, for the rows
- A second categorical column - this will be for the columns
- A numeric column
- Aggregation method

In [11]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,25.57,16.941386,19.076807,19.002803,20.518657,20.466667,NaN
2,NaN,17.904989,17.855770,17.359076,17.927913,17.192379,17.401355


In [12]:
df.groupby(['VendorID', 'passenger_count'])['total_amount'].mean()

VendorID  passenger_count
1         0                  25.570000
          1                  16.941386
          2                  19.076807
          3                  19.002803
          4                  20.518657
          5                  20.466667
2         1                  17.904989
          2                  17.855770
          3                  17.359076
          4                  17.927913
          5                  17.192379
          6                  17.401355
Name: total_amount, dtype: float64

In [13]:
df.pivot(index='VendorID', columns='passenger_count', values='total_amount')

ValueError: Index contains duplicate entries, cannot reshape

In [15]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='mean')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,25.57,16.941386,19.076807,19.002803,20.518657,20.466667,NaN
2,NaN,17.904989,17.855770,17.359076,17.927913,17.192379,17.401355


In [16]:
df.pivot_table(index='VendorID', columns='passenger_count', values='total_amount',
              aggfunc='max')

passenger_count,0,1,2,3,4,5,6
VendorID,,,,,,,
1,36.39,252.35,102.35,73.55,73.84,27.30,NaN
2,NaN,137.59,138.84,74.46,72.92,102.11,83.12


In [18]:
# what if we want both mean + standard deviation?
df.pivot_table(columns='VendorID', index='passenger_count', values='total_amount',
              aggfunc=['mean', 'std'])

mean                   std           
VendorID                 1          2          1          2
passenger_count                                            
0                25.570000        NaN  15.301791        NaN
1                16.941386  17.904989  15.369459  15.167490
2                19.076807  17.855770  16.049125  15.233530
3                19.002803  17.359076  15.389968  12.540837
4                20.518657  17.927913  15.845568  14.630713
5                20.466667  17.192379   6.751543  14.064202
6                      NaN  17.401355        NaN  13.363827

In [19]:
# what if we want to look at more than one value column?
df.pivot_table(columns='VendorID', index='passenger_count', values=['total_amount', 'trip_distance'],
              aggfunc='mean')

total_amount            trip_distance          
VendorID                   1          2             1         2
passenger_count                                                
0                  25.570000        NaN      4.600000       NaN
1                  16.941386  17.904989      2.956456  3.262967
2                  19.076807  17.855770      3.452027  3.328849
3                  19.002803  17.359076      3.588535  3.187189
4                  20.518657  17.927913      3.952239  3.440522
5                  20.466667  17.192379      4.933333  3.172553
6                        NaN  17.401355           NaN  3.170976

In [23]:
# what if we want to look at more than one category for columns?
df.pivot_table(columns=['VendorID', 'payment_type'], index='passenger_count', values='total_amount',
              aggfunc='mean')

VendorID                 1                                        2  \
payment_type             1          2          3       4          1   
passenger_count                                                       
0                25.570000        NaN        NaN     NaN        NaN   
1                19.673469  12.698253  13.215172  11.675  20.147390   
2                21.338436  16.006464  42.480000     NaN  20.623066   
3                19.225065  18.763418        NaN  20.800  18.548926   
4                24.138387  17.401667        NaN     NaN  21.539123   
5                      NaN  20.466667        NaN     NaN  19.634750   
6                      NaN        NaN        NaN     NaN  18.980288   

VendorID                              
payment_type             2    3    4  
passenger_count                       
0                      NaN  NaN  NaN  
1                14.510453 -5.3 -7.8  
2                13.905152  NaN  NaN  
3                15.586200  NaN  NaN  
4                14.378966  NaN  NaN  
5                13.225076  NaN  NaN  
6                15.361491  NaN  NaN

In [22]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RateCodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [24]:
# everything can be a list!
df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

mean                                     \
                             total_amount            trip_distance             
VendorID                                1          2             1         2   
passenger_count payment_type                                                   
0               1               25.570000        NaN      4.600000       NaN   
1               1               19.673469  20.147390      3.346904  3.486413   
                2               12.698253  14.510453      2.358766  2.925338   
                3               13.215172  -5.300000      2.000000  0.465000   
                4               11.675000  -7.800000      1.925000  0.890000   
2               1               21.338436  20.623066      3.461350  3.751722   
                2               16.006464  13.905152      3.346388  2.725152   
                3               42.480000        NaN     11.700000       NaN   
3               1               19.225065  18.548926      3.110390  3.278926   
                2               18.763418  15.586200      4.054430  3.050500   
                4               20.800000        NaN      3.600000       NaN   
4               1               24.138387  21.539123      4.348387  4.059298   
                2               17.401667  14.378966      3.611111  2.832414   
5               1                     NaN  19.634750           NaN  3.518750   
                2               20.466667  13.225076      4.933333  2.610203   
6               1                     NaN  18.980288           NaN  3.178606   
                2                     NaN  15.361491           NaN  3.161118   

                                      std                                     
                             total_amount            trip_distance            
VendorID                                1          2             1         2  
passenger_count payment_type                                                  
0               1               15.301791        NaN      4.666905       NaN  
1               1               17.401710  16.330567      4.120243  4.238130  
                2               10.068471  12.397838      3.495789  3.973524  
                3               15.363039   2.828427      3.300000  0.657609  
                4                6.750000        NaN      1.960230       NaN  
2               1               17.341582  16.457943      4.038718  4.584112  
                2               13.427005  12.278545      4.502270  3.465173  
                3               27.061360        NaN      9.134002       NaN  
3               1               14.268286  13.073727      3.406595  3.529512  
                2               16.588856  11.539061      5.085722  3.363113  
                4                     NaN        NaN           NaN       NaN  
4               1               18.045435  17.046583      4.541429  4.831724  
                2               13.143542  10.806051      3.871048  4.002527  
5               1                     NaN  15.734323           NaN  4.261502  
                2                6.751543   9.604873      2.926317  3.393812  
6               1                     NaN  13.856405           NaN  3.499148  
                2                     NaN  12.447223           NaN  4.083165

In [25]:
pt = df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

In [29]:
# passenger_count 4
# payment_type 1

# function mean
# column total_amount
# vendor ID 2

pt.loc[
   (4, 1)   # row selector
,
   # column selector
   ('mean', 'total_amount', 2)
]

21.539122807017545

In [30]:
# I want payment type 1 for all passenger counts

pt.xs(1, level='payment_type')

mean                                             std  \
                total_amount            trip_distance           total_amount   
VendorID                   1          2             1         2            1   
passenger_count                                                                
0                  25.570000        NaN      4.600000       NaN    15.301791   
1                  19.673469  20.147390      3.346904  3.486413    17.401710   
2                  21.338436  20.623066      3.461350  3.751722    17.341582   
3                  19.225065  18.548926      3.110390  3.278926    14.268286   
4                  24.138387  21.539123      4.348387  4.059298    18.045435   
5                        NaN  19.634750           NaN  3.518750          NaN   
6                        NaN  18.980288           NaN  3.178606          NaN   

                                                    
                           trip_distance            
VendorID                 2             1         2  
passenger_count                                     
0                      NaN      4.666905       NaN  
1                16.330567      4.120243  4.238130  
2                16.457943      4.038718  4.584112  
3                13.073727      3.406595  3.529512  
4                17.046583      4.541429  4.831724  
5                15.734323           NaN  4.261502  
6                13.856405           NaN  3.499148

In [31]:
# we can use xs on columns, too!

pt.xs('trip_distance', level=1, axis='columns')

mean                 std          
VendorID                              1         2         1         2
passenger_count payment_type                                         
0               1              4.600000       NaN  4.666905       NaN
1               1              3.346904  3.486413  4.120243  4.238130
                2              2.358766  2.925338  3.495789  3.973524
                3              2.000000  0.465000  3.300000  0.657609
                4              1.925000  0.890000  1.960230       NaN
2               1              3.461350  3.751722  4.038718  4.584112
                2              3.346388  2.725152  4.502270  3.465173
                3             11.700000       NaN  9.134002       NaN
3               1              3.110390  3.278926  3.406595  3.529512
                2              4.054430  3.050500  5.085722  3.363113
                4              3.600000       NaN       NaN       NaN
4               1              4.348387  4.059298  4.541429  4.831724
                2              3.611111  2.832414  3.871048  4.002527
5               1                   NaN  3.518750       NaN  4.261502
                2              4.933333  2.610203  2.926317  3.393812
6               1                   NaN  3.178606       NaN  3.499148
                2                   NaN  3.161118       NaN  4.083165

# Exercise: Olympic pivot tables

1. Load the Olympic data (`olympic_athlete_events.csv`) into a data frame. Keep only the rows for the teams from : Israel, US, UK, France, India, China.
2. Create a pivot table of teams vs. sport, average height, where the year is equal to or after 1980, and the sport has `i` in its name.
3. Show a pivot table of teams vs. sport, min + max weight. 
4. Show a pivot table of teams vs. sport, min and max weight + height.
5. Show a pivot table of years vs. teams, number of competitors.  Retrieve the number for Israel in each year.

In [32]:
pt = df.pivot_table(columns='VendorID', index=['passenger_count', 'payment_type'],
               values=['total_amount', 'trip_distance'],
              aggfunc=['mean', 'std'])

In [34]:
df = pd.read_csv('olympic_athlete_events.csv',
                usecols=['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'])
df.columns

Index(['Name', 'Age', 'Height', 'Weight', 'Team', 'Year', 'Sport'], dtype='object')

In [ ]:
df['']